# StackOverflow - Exploratory Data Analysis

Perform an Exploratory Data Analysis over the data included in the BigQuery dataset `bigquery-public-data.stackoverflow`, in order to uncover interesting patterns and potential use cases within the data.

# Setup Notebook

## Imports

In [23]:
# Import Standard Libraries
import os
from pathlib import Path
from dynaconf import Dynaconf

# Import Package Modules
from src.bigquery_connector.bigquery_connector import BigQueryConnector
from src.types import BigQueryClientConfig

## Define Configurations

In [19]:
# Setup root path
root_path = Path(os.getcwd()).parents[2]

In [20]:
# Read configuration
config = Dynaconf(
    settings_files=[
        root_path / 'configuration' / 'settings.toml'
    ],
    environments=True, 
    env='eda'
)

In [21]:
# Retrieve configurations
bigquery_client_config = config['bigquery_client']
relevant_users_config = config['relevant_users']

# Read Data

In [24]:
# Instance BigQuery Connector
bigquery_connector = BigQueryConnector(BigQueryClientConfig(**bigquery_client_config))

[10/30/2024 15:32:57 - BigQueryConnector] INFO - _set_client - Set the BigQuery client


In [26]:
# Read data from BigQuery
relvant_users = bigquery_connector.read_from_query_config(relevant_users_config)

[10/30/2024 15:40:04 - BigQueryConnector] INFO - read_from_query_config - Reading query file: queries/stackoverflow/eda/users/relevant_users.sql


FileNotFoundError: Unable to locate file: queries/stackoverflow/eda/users/relevant_users.sql